# LinearRegression from Scratch


# 구현할 것
- 공부시간과 성적간의 관계를 모델링한다.
    - **머신러닝 모델(모형)이란** 수집한 데이터를 기반으로 입력값(Feature)와 출력값(Target)간의 관계를 하나의 공식으로 정의한 함수이다. 그 공식을 찾는 과정을 **모델링**이라고 한다.
    - 이 예제에서는 공부한 시험시간으로 점수를 예측하는 모델을 정의한다.
    - 입력값과 출력값 간의 관계를 정의할 수있는 다양한 함수(공식)이 있다. 여기에서는 딥러닝과 관계가 있는 **Linear Regression** 을 사용해본다.

# 데이터 확인
- 입력데이터: 공부시간
- 출력데이터: 성적

|공부시간|점수|
|-|-|
|1|20|
|2|40|
|3|60|

우리가 수집한 공부시간과 점수 데이터를 바탕으로 둘 간의 관계를 식으로 정의 할 수 있으면 **내가 몇시간 공부하면 점수를 얼마 받을 수 있는지 예측할 수 있게 된다.**   

- X가 시간 y가 점수!

수집한 데이터를 기반으로 앞으로 예측할 수있는 모형을 만드는 것이 머신러닝 모델링이다.

  

## 학습(훈련) 데이터셋 만들기
- 모델을 학습시키기 위한 데이터셋을 구성한다.
- 입력데이터와 출력데이터을 각각 다른 행렬로 구성한다.
- 하나의 데이터 포인트의 입력/출력 값은 같은 index에 정의한다.

### 선형회귀 (Linear Regression)
- Feature들의 가중합을 이용해 Target을 추정한다.
- Feature에 곱해지는 가중치(weight)들은 각 Feature가 Target 얼마나 영향을 주는지 영향도가 된다.
    - 음수일 경우는 target값을 줄이고 양수일 경우는 target값을 늘린다.
    - 가중치가 0에 가까울 수록 target에 영향을 주지 않는 feature이고 0에서 멀수록 target에 많은 영향을 준다.
- 모델 학습과정에서 가장 적절한 Feature의 가중치를 찾아야 한다.
      

\begin{align}
&\large \hat{y} = W\cdot X + b\\
&\small \hat{y}: \text{모델추정값}\\
&\small W: \text{가중치}\\
&\small X: \text{Feature(입력값)}\\
&\small b: \text{bias(편향)}
\end{align}



## Train dataset 구성
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature의 행은 관측치(개별 데이터)를 열을 Feature(특성, 변수)를 표현한다. 이 문제에서는 `공부시간` 1개의 변수를 가진다.
- Target은 모델이 예측할 대상으로 행은 개별 관측치, 열은 각 항목에 대한 정답으로 구성한다.   
  이 문제에서 예측할 항목은 `시험점수` 한개이다.

In [1]:
import torch

In [2]:
study_time = [[1],[2],[3]] # 2차원 리스트로 생성
score = [[20], [30], [40]]

# dataset 구성 -> torch.Tensor
X_train = torch.tensor(study_time, dtype=torch.float32)
y_train = torch.tensor(score, dtype=torch.float32)
print(X_train.size(), y_train.size())

torch.Size([3, 1]) torch.Size([3, 1])


## 파라미터 (weight, bias) 정의
- 학습대상/최적화 대상

In [4]:
# X * weight + bias
torch.manual_seed(0)

# weight/bias 정의
## feature는 1개라서 w값도 1개를 갖고, x는 2x1이므로 shape이 1로 시작하여야하므로 w는 1x1 shape을 갖도록 해야한다.(행렬 내적곱)
## weight는 표준정규분포의 랜덤값을 사용.
weight = torch.randn(1,1,requires_grad=True)
bias = torch.randn(1,requires_grad=True)

print("초기 파라미터")
print(weight)
print(bias)

초기 파라미터
tensor([[1.5410]], requires_grad=True)
tensor([-0.2934], requires_grad=True)


In [5]:
# 모델 정의
def linear_model(X):
  return X @ weight + bias # 2x1 @ 1x1

# 손실 함수(loss) 정의 -> MSE
def loss_fn(y_pred, y): # (y예측, y정답) 파이토치에서는 예측값을 먼저 적는다.
  return torch.mean((y_pred - y)**2)

In [6]:
y_pred = linear_model(X_train)
y_pred
# 오차 확인, y_pred에 대한 X의 도함수를 grad_fn로 저장

tensor([[1.2476],
        [2.7886],
        [4.3296]], grad_fn=<AddBackward0>)

In [7]:
loss_fn(y_pred, y_train) # 손실함수값 확인
# 출력: tensor(788.1655, grad_fn=<MeanBackward0>)

tensor(788.1655, grad_fn=<MeanBackward0>)

### 모델링

### 학습
1. 모델을 이용해 추정한다.
   - pred = model(input)
1. loss를 계산한다.
   - loss = loss_fn(pred, target)
1. 계산된 loss를 파라미터에 대해 미분하여 계산한 gradient 값을 각 파라미터에 저장한다.
   - loss.backward()
1. optimizer를 이용해 파라미터를 update한다.
   - optimizer.step()  
1. 파라미터의 gradient(미분값)을 0으로 초기화한다.
   - optimizer.zero_grad()
- 위의 단계를 반복한다.   

In [9]:
# 학습

LEARNING_RATE = 0.01
EPOCHS = 2000 # 반복횟수. 전체 train dataset을 한 번 다 학습하면 1회 epoch이라고 한다.

for epoch in range(EPOCHS):
    # 1. 모델을 이용한 추론
    pred = linear_model(X_train)

    # 2. loss 계산
    loss = loss_fn(pred, y_train)

    # 3. gradient 계산
    loss.backward() # weight/bias의 grad값이 계산되서 각각의 grad 속성에 저장.

    # 4. parameter(weight/bias) 업데이트
    weight.data = weight.data - weight.grad * LEARNING_RATE
    bias.data = bias.data - bias.grad * LEARNING_RATE

    # 5. 파라미터 grad값 초기화
    weight.grad = None
    bias.grad = None
    if epoch % 100 == 0 or epoch == EPOCHS - 1: # 조건문의 의미: 100번에 한번씩, 마지막 에폭이라면.
        print(f"[{epoch}/{EPOCHS}] loss: {loss.item()}")

# 아래 결과보면, 오차가 788에서 점점 작아진다. 작아지는 폭도 점점 작아지는데, grad값이 점점 작아지기 때문(변화율이 점점 줄어드는 것을 확인할 수 있다.)

[0/2000] loss: 0.0002866916183847934
[100/2000] loss: 0.00017719365132506937
[200/2000] loss: 0.00010951198055408895
[300/2000] loss: 6.768395542167127e-05
[400/2000] loss: 4.183211058261804e-05
[500/2000] loss: 2.5878276574076153e-05
[600/2000] loss: 1.6003566997824237e-05
[700/2000] loss: 9.907454113999847e-06
[800/2000] loss: 6.132206181064248e-06
[900/2000] loss: 3.8017424230929464e-06
[1000/2000] loss: 2.3568343294755323e-06
[1100/2000] loss: 1.4622297612731927e-06
[1200/2000] loss: 9.11352515231556e-07
[1300/2000] loss: 5.673452392329636e-07
[1400/2000] loss: 3.569390685242979e-07
[1500/2000] loss: 2.213406986584232e-07
[1600/2000] loss: 1.3866035430964985e-07
[1700/2000] loss: 8.980472188113708e-08
[1800/2000] loss: 5.144102033227682e-08
[1900/2000] loss: 3.414849558680544e-08
[1999/2000] loss: 2.229838536038642e-08


In [10]:
# 추론
pred2 = linear_model(X_train)
pred2

tensor([[19.9998],
        [30.0000],
        [40.0001]], grad_fn=<AddBackward0>)

In [11]:
weight, bias

(tensor([[10.0002]], requires_grad=True), tensor([9.9996], requires_grad=True))

 다중 입력, 다중 출력
- 다중입력: Feature가 여러개인 경우
- 다중출력: Output 결과가 여러개인 경우

다음 가상 데이터를 이용해 사과와 오렌지 수확량을 예측하는 선형회귀 모델을 정의한다.  
[참조](https://www.kaggle.com/code/aakashns/pytorch-basics-linear-regression-from-scratch)


|온도(F)|강수량(mm)|습도(%)|사과생산량(ton)|오렌지생산량|
|-|-|-|-:|-:|
|73|67|43|56|70|
|91|88|64|81|101|
|87|134|58|119|133|
|102|43|37|22|37|
|69|96|70|103|119|

```
사과수확량  = w11 * 온도 + w12 * 강수량 + w13 * 습도 + b1
오렌지수확량 = w21 * 온도 + w22 * 강수량 + w23 *습도 + b2
```

- `온도`, `강수량`, `습도` 값이 **사과**와, **오렌지 수확량**에 어느정도 영향을 주는지 가중치를 찾는다.
    - 모델은 사과의 수확량, 오렌지의 수확량 **두개의 예측결과를 출력**해야 한다.
    - 사과에 대해 예측하기 위한 weight 3개와 오렌지에 대해 예측하기 위한 weight 3개 이렇게 두 묶음, 총 6개의 weight를 정의하고 학습을 통해 가장 적당한 값을 찾는다.
        - `개별 과일를 예측하기 위한 weight들 @ feature들` 의 계산 결과를  **Node, Unit, Neuron** 이라고 한다.
        - 두 과일에 대한 Unit들을 묶어서 **Layer** 라고 한다.
- 목적은 우리가 수집한 train 데이터셋을 이용해 **정확한 예측을 위한 weight와 bias 들**을 찾는 것이다.

## Train Dataset
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature의 행은 관측치(개별 데이터)를 열을 Feature(특성, 변수)를 표현한다. 이 문제에서는 `온도, 강수량, 습도` 세개의 변수를 가진다.
- Target은 모델이 예측할 대상으로 행은 개별 관측치, 열은 각 항목에 대한 정답으로 구성한다. 이 문제에서 예측할 항목은 `사과수확량, 오렌지 수확량` 2개의 값이다.

In [12]:
#  input: 생산환경 (temp, rainfall, humidity) : (5, 3)
environs = [
    [73, 67, 43],
    [91, 88, 64],
    [87, 134, 58],
    [102, 43, 37],
    [69, 96, 70]
]

# Targets: 생산량 - (apples, oranges) - (5, 2)
apple_orange_output = [
    [56, 70],
    [81, 101],
    [119, 133],
    [22, 37],
    [103, 119]
]

In [15]:
import torch
# Dataset을 torch.Tensor로 생성
X = torch.tensor(environs, dtype=torch.float32)
y = torch.tensor(apple_orange_output, dtype=torch.float32)
X.shape, y.shape # X.shape: 속성이 3개, y.shape: 사과생산량, 오렌지생산량 2개

(torch.Size([5, 3]), torch.Size([5, 2]))

In [16]:
X

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [17]:
y

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## weight와 bias
- weight: 각 feature들이 생산량에 영향을 주었는지의 가중치로 feature에 곱해줄 값.
    - 사과, 오렌지의 생산량을 구해야 하므로 가중치가 두개가 된다.
    - weight의 shape: `(3, 2)`
- bias는 모든 feature들이 0일때 생산량이 얼마일지를 나타내는 값으로 feature와 weight간의 가중합 결과에 더해줄 값이다.
    - 사과, 오렌지의 생산량을 구하므로 bias가 두개가 된다.
    - bias의 shape: `(2, )`

### Linear Regression model
모델은 weights `w`와 inputs `x`의 내적(dot product)한 값에 bias `b`를 더하는 함수.

$$
\hspace{2.5cm} X \hspace{1.1cm} \cdot \hspace{1.2cm} W \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\cdot
%
\left[ \begin{array}{cc}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b_{1} & b_{2} \\
b_{1} & b_{2} \\
\vdots & \vdots \\
b_{1} & b_{2} \\
\end{array} \right]
$$


<center style="font-size:0.9em">
$w_{11},\,w_{12},\,w_{13}$: 사과 생산량 계산시 각 feature들(생산환경)에 곱할 가중치   <br>
$w_{21},\,w_{22},\,w_{23}$: 오렌지 생산량 계산시 각 feature들(생산환경)에 곱할 가중치    
</center>

<center>
<img src="https://raw.githubusercontent.com/kgmyhGit/image_resource/main/deeplearning/figures/3_unit_layer.png">
</center>

In [18]:
# weight/bias 를 정의 -> 초기값은 random 값을 이용해서 생성.
weight = torch.randn(3, 2, requires_grad=True) # 3은 feature 입력값 개수, 2는 예측할 값의 개수
bias = torch.randn(2, requires_grad=True) # 2는 예측할 값의 개수

weight.size(), bias.size()
# weight: (3:input feature개수  ,  2:output 개수)
# bias  : (2:output 개수, )

(torch.Size([3, 2]), torch.Size([2]))

In [19]:
#random으로 나온 초기값 weight
weight

tensor([[-2.1788,  0.5684],
        [-1.0845, -1.3986],
        [ 0.4033,  0.8380]], requires_grad=True)

In [20]:
#random으로 나온 초기값 bias
bias

tensor([-0.7193, -0.4033], requires_grad=True)

In [21]:
X

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [22]:
# 한번 학습(최적화)
## 추론
pred = X @ weight + bias

In [23]:
pred # 예측한 값

tensor([[-215.0900,  -16.5786],
        [-268.6129,  -18.1188],
        [-312.2058,  -89.7561],
        [-254.6664,   28.4440],
        [-226.9356,  -36.7849]], grad_fn=<AddBackward0>)

In [24]:
y # 정답

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [25]:
## loss 계산(MSE)
loss = torch.mean((pred - y)**2) # 전체 추론한 결과의 평균오차를 계산.
loss

tensor(66270.6719, grad_fn=<MeanBackward0>)

In [26]:
# loss를 가지고 파라미터들(weight들, bias들)의 gradient 계산.
loss.backward()

In [30]:
weight.data, weight.grad

(tensor([[-2.1788,  0.5684],
         [-1.0845, -1.3986],
         [ 0.4033,  0.8380]]),
 tensor([[-28020.9551,  -9632.3398],
         [-30056.2031, -12291.1592],
         [-18474.8359,  -7097.5703]]))

In [31]:
bias.data, bias.grad

(tensor([-0.7193, -0.4033]), tensor([-331.7021, -118.5589]))

In [32]:
# 파라미터 업데이트
lr = 0.00001 # learning rate
weight.data = weight.data - lr * weight.grad
bias.data = bias.data - lr * bias.grad

In [33]:
weight.data

tensor([[-1.8986,  0.6648],
        [-0.7840, -1.2757],
        [ 0.5881,  0.9090]])

In [34]:
## 업데이트된 파라미터로 추정 -> loss 계산
pred2 = X @ weight + bias
loss2 = torch.mean((pred2 - y)**2)

In [35]:
print(loss.item(), loss2.item())
# 아래의 loss는 사과와 오렌지에 관한 loss 정보가 함께 있다.
# 사과와 오렌지에 대한 각각의 loss를 구할 수 있다. 이때는 사과와 오렌지에 대한 중요도가 다를때 나눠서 구하기도 한다.
# 결국엔 total loss를 구해야하기는 하지만, 따로 구하는건 나중에 배울 예정이고 지금은 전체 loss를 구하여 진행하는 것만 배움

66270.671875 45108.19921875


##  모델링

In [36]:
weight = torch.randn(3, 2, requires_grad=True)
bias = torch.randn(2, requires_grad=True)

## 모델 정의
def model(X):
    return X @ weight + bias

## loss 함수(MSE)
def loss_fn(pred, y):
    return torch.mean((pred - y)**2) # 전체 오차의 평균.

In [37]:
X.shape

torch.Size([5, 3])

In [38]:
epochs = 5000
lr = 0.00001  # 1e-5
for epoch in range(epochs):
    # 1. 추론
    pred = model(X)

    # 2. loss 계산
    loss = loss_fn(pred, y)

    # 3. 파라미터 들의 gradient 계산
    loss.backward()

    # 4. 파라미터 업데이트
    weight.data = weight.data - lr * weight.grad
    bias.data = bias.data - lr * bias.grad

    # 5. gradient 초기화
    weight.grad = None
    bias.grad = None

    ## 100 epoch, 마지막 epoch에서 loss를 출력 => 학습 과정 log를 출력
    if epoch % 100 == 0 or epoch == epochs-1:
        print(f"[{epoch+1:04d}/{epochs}] - {loss.item():.5f}")


  # loss가 점점 줄어드는것 확인!

[0001/5000] - 36143.92969
[0101/5000] - 180.45209
[0201/5000] - 88.39181
[0301/5000] - 56.18957
[0401/5000] - 41.86906
[0501/5000] - 33.46677
[0601/5000] - 27.47202
[0701/5000] - 22.77247
[0801/5000] - 18.95049
[0901/5000] - 15.80123
[1001/5000] - 13.19460
[1101/5000] - 11.03372
[1201/5000] - 9.24149
[1301/5000] - 7.75476
[1401/5000] - 6.52136
[1501/5000] - 5.49811
[1601/5000] - 4.64920
[1701/5000] - 3.94491
[1801/5000] - 3.36064
[1901/5000] - 2.87589
[2001/5000] - 2.47372
[2101/5000] - 2.14008
[2201/5000] - 1.86328
[2301/5000] - 1.63365
[2401/5000] - 1.44314
[2501/5000] - 1.28509
[2601/5000] - 1.15396
[2701/5000] - 1.04519
[2801/5000] - 0.95493
[2901/5000] - 0.88006
[3001/5000] - 0.81794
[3101/5000] - 0.76641
[3201/5000] - 0.72365
[3301/5000] - 0.68818
[3401/5000] - 0.65875
[3501/5000] - 0.63434
[3601/5000] - 0.61408
[3701/5000] - 0.59728
[3801/5000] - 0.58334
[3901/5000] - 0.57177
[4001/5000] - 0.56218
[4101/5000] - 0.55422
[4201/5000] - 0.54761
[4301/5000] - 0.54213
[4401/5000] - 0.

In [39]:
# 새로운 데이터로 추론
p = model(X)
p

tensor([[ 57.0795,  70.3786],
        [ 82.2048, 100.5718],
        [118.8339, 133.0717],
        [ 21.1284,  37.0420],
        [101.7800, 119.0247]], grad_fn=<AddBackward0>)

In [40]:
y # 새로운 데이터로 추론한 p의 값들과 정답인 y값들의 차이를 보면 차이가 매우 적어졌다!

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [41]:
new_x = torch.tensor([68, 82, 56], dtype=torch.float32)
new_x = new_x.unsqueeze(dim=0)
new_x.shape # shape은 3으로 맞춰줘야한다. 하나의 지역에 대해서만 보려는 것이므로 0축은 1이 들어가도록 unsqueeze(dim=0) 넣어주고, 1축은 3이 들어가야함

torch.Size([1, 3])

In [42]:
model(new_x)

tensor([[80.6958, 95.5601]], grad_fn=<AddBackward0>)

# pytorch built-in 모델을 사용해 Linear Regression 구현

In [43]:
inputs = torch.tensor(
    [[73, 67, 43],
     [91, 88, 64],
     [87, 134, 58],
     [102, 43, 37],
     [69, 96, 70]], dtype=torch.float32)

In [44]:
targets = torch.tensor(
    [[56, 70],
    [81, 101],
    [119, 133],
    [22, 37],
    [103, 119]], dtype=torch.float32)

## torch.nn.Linear
Pytorch는 torch.nn.Linear 클래스를 통해 Linear Regression 모델을 제공한다.  
torch.nn.Linear에 입력 feature의 개수와 출력 값의 개수를 지정하면 random 값으로 초기화한 weight와 bias들을 생성해 모델을 구성한다. (ex. 위에서의 경우, 3과 2를 지정해준것을 말함)
- 위에서 만든 y = w*X + b 에 관한 모델을 만드는 객체
- `torch.nn.Linear(input feature의 개수 , output 값의 개수)`

## Optimizer와 Loss 함수 정의
- **Optimizer**: 계산된 gradient값을 이용해 파라미터들을 업데이트 하는 함수
- **Loss 함수**: 정답과 모델이 예측한 값사이의 차이(오차)를 계산하는 함수.
  - 모델을 최적화하는 것은 이 함수의 값을 최소화하는 것을 말한다.
- `torch.optim` 모듈에 다양한 Optimizer 클래스가 구현되있다.
- `torch.nn` 또는 `torch.nn.functional` 모듈에 다양한 Loss 함수가 제공된다.

In [45]:
# 선형회귀 모델을 정의. torch.nn.Linear 클래스
import torch
import torch.nn as nn

model = nn.Linear(3, 2)  # 3: input feature 개수, 2: output 수

In [46]:
# loss 함수
loss_fn = torch.nn.MSELoss()  # 클래스
# loss_fn = torch.nn.functional.mse_loss # 함수

In [48]:
# optimizer (torch.optim 모듈에 정의): weight.data = weight.data - lr * weight.grad
# optimizer : 파라미터를 업데이트하는 로직 or 초기화하는 로직을 가지고 있다.
# 많은 optimizer 중에서 SGD를 쓰는 것. SGD가 가장 기본의 것이다.
optimizer = torch.optim.SGD(
    model.parameters(), # 최적화 대상 파라미터들을 model에서 조회해서 전달. optimizer와 파라미터들을 함께 넘겨주어 진행함. 우리가 아까 직접계산하던걸 optimizer에게 맡기는 것.
    lr = 0.00001,       # Learning Rage
)

In [49]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3370,  0.4963,  0.2576],
         [ 0.2798,  0.0304, -0.2960]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0977, -0.5391], requires_grad=True)]

## Model Train

In [50]:
epochs = 5000

for epoch in range(epochs):

    # 추론
    pred = model(inputs)

    # loss 계산
    loss = loss_fn(pred, targets) # torch.nn.functional.mse_loss(pred, targets) # (모델추정값, 정답)

    # gradient 계산
    loss.backward()

    # 파라미터 업데이트: optimizer.step()
    optimizer.step()

    # 파라미터 초기화 w.grad=None, b.grad=None
    optimizer.zero_grad()

    # 현재 epoch 학습 결과를 log로 출력
    if epoch % 100 == 0 or epoch == epochs-1:
        print(f"[{epoch+1:04d}/{epochs}] - {loss.item()}")

[0001/5000] - 5417.5205078125
[0101/5000] - 139.60592651367188
[0201/5000] - 60.372108459472656
[0301/5000] - 34.58899688720703
[0401/5000] - 24.409469604492188
[0501/5000] - 19.110050201416016
[0601/5000] - 15.596234321594238
[0701/5000] - 12.928831100463867
[0801/5000] - 10.785379409790039
[0901/5000] - 9.026604652404785
[1001/5000] - 7.572966575622559
[1101/5000] - 6.368545055389404
[1201/5000] - 5.3697309494018555
[1301/5000] - 4.5411906242370605
[1401/5000] - 3.853886842727661
[1501/5000] - 3.2836735248565674
[1601/5000] - 2.810603141784668
[1701/5000] - 2.4181599617004395
[1801/5000] - 2.092557430267334
[1901/5000] - 1.822432518005371
[2001/5000] - 1.5983161926269531
[2101/5000] - 1.4123988151550293
[2201/5000] - 1.2581546306610107
[2301/5000] - 1.130189061164856
[2401/5000] - 1.0240285396575928
[2501/5000] - 0.935954749584198
[2601/5000] - 0.8628835678100586
[2701/5000] - 0.8022648692131042
[2801/5000] - 0.7519651055335999
[2901/5000] - 0.710246741771698
[3001/5000] - 0.67563104

In [52]:
# 추론 => gradient 계산을 할 필요가 없다. ==> grad_fn을 만들 필요가 없다. 그래서 with torch.no_grad() 블록에서 추론 작업을 실행한다.
with torch.no_grad():
    pred = model(inputs)

In [53]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [54]:
pred

tensor([[ 57.1837,  70.2264],
        [ 82.1790, 100.6659],
        [118.7269, 133.0993],
        [ 21.0897,  37.0620],
        [101.8865, 118.9974]])

In [55]:
# 학습 로직을 함수 구현
def train(inputs, targets, epochs, model, loss_fn, optimizer):

    for epoch in range(epochs):

        # 추론
        pred = model(inputs)

        # loss 계산
        loss = loss_fn(pred, targets) # torch.nn.functional.mse_loss(pred, targets) # (모델추정값, 정답)

        # gradient 계산
        loss.backward()
        # 파라미터 업데이트: optimizer.step()

        optimizer.step()
        # 파라미터 초기화 w.grad=None, b.grad=None
        optimizer.zero_grad()

        # 현재 epoch 학습 결과를 log로 출력
        if epoch % 100 == 0 or epoch == epochs-1:
            print(f"[{epoch+1:04d}/{epochs}] - {loss.item()}")

In [56]:
model = nn.Linear(3, 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [57]:
train(inputs, targets, 5000, model, nn.functional.mse_loss, optimizer)

[0001/5000] - 12192.0791015625
[0101/5000] - 3.2720978260040283
[0201/5000] - 0.9513593912124634
[0301/5000] - 0.5959173440933228
[0401/5000] - 0.54143887758255
[0501/5000] - 0.5330857634544373
[0601/5000] - 0.5317986607551575
[0701/5000] - 0.5315942764282227
[0801/5000] - 0.5315550565719604
[0901/5000] - 0.5315428972244263
[1001/5000] - 0.5315331220626831
[1101/5000] - 0.5315253734588623
[1201/5000] - 0.5315181016921997
[1301/5000] - 0.53150874376297
[1401/5000] - 0.5315012335777283
[1501/5000] - 0.531493604183197
[1601/5000] - 0.5314861536026001
[1701/5000] - 0.5314829349517822
[1801/5000] - 0.5314752459526062
[1901/5000] - 0.5314642786979675
[2001/5000] - 0.5314569473266602
[2101/5000] - 0.5314476490020752
[2201/5000] - 0.5314388871192932
[2301/5000] - 0.5314317345619202
[2401/5000] - 0.5314244031906128
[2501/5000] - 0.5314170718193054
[2601/5000] - 0.5314087271690369
[2701/5000] - 0.531402587890625
[2801/5000] - 0.5313918590545654
[2901/5000] - 0.5313825607299805
[3001/5000] - 0.53